In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_19420\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
base_url = "https://www.justwatch.com"

In [5]:
# Python3 Program to Create list 
# with integers within given range 
  
def createYears(r1, r2):
    return [item for item in range(r1, r2+1)]
      
genres = ['act','cmy',  'doc', 'fnt', 'hrr', 'msc', 'rms', 'spt', 'wsn', 'ani', 'crm', 'drm', 'hst', 'fml', 'trl', 'scf', 'war', 'rly']
years = createYears(1900, 2023)


search_links = []
for i in genres:
    for j in years:
        link = f"https://www.justwatch.com/us/movies?genres={i}&release_year_from={j}&release_year_until={j}"
        search_links.append(link)
        
len(search_links)

2232

In [6]:
titleLinks = []
for link in search_links[:100]:
    chrome_options = Options()  
    chromedriver = webdriver.Chrome(ChromeDriverManager().install())
    with chromedriver as browser:
        browser.get(link)
        time.sleep(5)

        last_height = browser.execute_script("return document.body.scrollHeight")
        SCROLL_PAUSE_TIME = 5
        while True:
                # Scroll down to bottom
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = browser.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
        page_source =browser.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        productlist = soup.find("div",{"listlayout":"Grid"})
        titles = productlist.find_all("div", {"class":"title-list-grid__item"})
        for title in titles:
                    title_link = title.find("a").get('href')
                    title_link = base_url+title_link
                    titleLinks.append(title_link)

C:\Users\user\AppData\Local\Temp\ipykernel_19420\2560210232.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chromedriver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
len(titleLinks)

9941

In [8]:
def get_detail(soup_object):
    labels  = []
    values = []
    table_info = soup_object.find("div", {"class":"title-info"})
    rows = table_info.find_all("div", {"class":"detail-infos"})
    for row in rows:
        label = row.find("div", {"class":"detail-infos__subheading"}).text.replace('\n',"")
        value = row.find("div", {"class":"detail-infos__value"}).text.replace('\n',"")
        labels.append(label)
        values.append(value)
        
    detail = {labels[i]: values[i] for i in range(len(labels))}
        
    return detail

def get_cast(soup_object):
    cast = []
    actors = soup_object.find_all("div", {"class":"title-credits__actor"})
    for actor in actors:
        actor_name = actor.text.replace('\n',"")
        cast.append(actor_name)
    return cast

class Pricing:
    def __init__(self, soup_object):
        self.soup_object = soup_object

    def get_stream_options(self):
        stream_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)


        return stream_list


    def get_rent_options(self):
        rent_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list

    def get_buy_options(self):
        buy_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list


In [9]:
def get_stream_options(soup_object):
        stream_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)
        return stream_list


def get_rent_options(soup_object):
        rent_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list
    
def get_buy_options(soup_object):
        buy_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list

In [10]:
def get_best(driver):
            driver.implicitly_wait(5)
            best = WebDriverWait(driver=driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div[3]/div/div[1]/div[1]/div/div[1]'))
            )
            best.click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_best = get_stream_options(hun)
            buy_best = get_buy_options(hun)
            rent_best = get_rent_options(hun)
            
            options = {"stream_best": stream_best,
                    "rent_best":rent_best,
                    "buy_best":buy_best}
            
            return options
        
def get_free(driver):
            driver.implicitly_wait(5)
            free = WebDriverWait(driver=driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div[3]/div/div[1]/div[1]/div/div[2]'))
            )
            free.click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_free = get_stream_options(hun)
            buy_free = get_buy_options(hun)
            rent_free = get_rent_options(hun)
            
            
            options = {"stream_free": stream_free,
                    "rent_free":rent_free,
                    "buy_free":buy_free}
        
            return options
        
def get_sd(driver):
            driver.implicitly_wait(5)
            sd = WebDriverWait(driver=driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div[3]/div/div[1]/div[1]/div/div[3]'))
            )
            sd.click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_sd = get_stream_options(hun)
            buy_sd = get_buy_options(hun)
            rent_sd = get_rent_options(hun)
            
            options = {"stream_sd": stream_sd,
                    "rent_sd":rent_sd,
                    "buy_sd":buy_sd}
        
            return options
        
def get_hd(driver):
        driver.implicitly_wait(5)
            
        hd = WebDriverWait(driver=driver, timeout=5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div[3]/div/div[1]/div[1]/div/div[4]'))
                )
            
        hd.click()
            
        time.sleep(5)
            
        page_source =driver.page_source
        
        hun=BeautifulSoup(page_source,'html.parser')
        stream_hd = get_stream_options(hun)
        buy_hd = get_buy_options(hun)
        rent_hd = get_rent_options(hun)
            
        options = {"stream_hd": stream_hd,
                    "rent_hd":rent_hd,
                    "buy_hd":buy_hd}
        
        return options
        
      
def get_4k(driver):   
        driver.implicitly_wait(5)
        fk = WebDriverWait(driver= driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div[3]/div/div[1]/div[1]/div/div[5]'))
            )
            
        fk.click()
            
        time.sleep(5)
            
        page_source =  driver.page_source
        hun=BeautifulSoup(page_source,'html.parser')
        stream_4k = get_stream_options(hun)
        buy_4k = get_buy_options(hun)
        rent_4k = get_rent_options(hun)       
            
        options = {"stream_4k": stream_4k,
                    "rent_4k":rent_4k,
                    "buy_4k":buy_4k}
        
        return options

In [15]:
def get_data(titleLinks):
    data = []
    for link in titleLinks[9841:]:
            driver = webdriver.Chrome(ChromeDriverManager().install())
            with driver as driver:
                driver.get(link)
                banner = WebDriverWait(driver=driver, timeout=5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div[2]/button[1]'))
                    )
                
                banner.click()
                
                time.sleep(1)
                try:
                    imdb_link = WebDriverWait(driver=driver, timeout=5).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[1]/div[2]/div/div[2]/a'))
                    )
               
                    imdb_link = imdb_link.get_attribute('href')
                except:
                    imdb_link = None
                
                page_source =  driver.page_source
                
                hun=BeautifulSoup(page_source,'html.parser')
                
                
                try:
                    detail = get_detail(hun)
                except:
                    detail = None
                    
                try:
                    title_name = hun.find("div", {"class":"title-block"})
                    title_name = title_name.find("h1").text.replace('\n',"")
                except:
                    title_name = None
                    
                try:
                    year = hun.find("span", {"class":"text-muted"}).text.replace('\n',"")
                except:
                    year = None
                try:   
                    prices_4k = get_4k(driver)
                except:
                    prices_4k = None
                    
                try:
                    prices_hd = get_hd(driver)
                except:
                    prices_hd = None
                
                try:
                    prices_sd = get_sd(driver)
                except:
                    prices_sd = None
                
                try:
                    prices_free = get_free(driver)
                except:
                    prices_free = None
                
                try:
                    prices_best = get_best(driver)
                except:
                    prices_best = None
        
        
                title_url = link
                
                tv = {"title":title_name, "year":year, "detail":detail, "4k prices":prices_4k, "hd prices":prices_hd, "sd prices":prices_sd, "free":prices_free, "best prices":prices_best,"imdb_link":imdb_link,"tite url": title_url}
        
                data.append(tv)
        
    return  data
        
        
        

In [16]:
Data = get_data(titleLinks)

C:\Users\user\AppData\Local\Temp\ipykernel_19420\270797039.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


def get_synopsis(soup_object):
    driver.get(soup_object)       
    synopsis = driver.find_element(By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/p').text
    return synopsis

get_rent_options(soup)

In [17]:
import pandas as pd
df = pd.DataFrame.from_dict(Data)
df

,title,year,detail,4k prices,hd prices,sd prices,free,best prices,imdb_link,tite url
0,In the Red,(1999),"{'Rating': ' 65% 3.2 ', 'Genres': 'Action & A...",None,None,None,None,None,https://www.imdb.com/title/tt0213724/?ref_=ref...,https://www.justwatch.com/us/movie/in-the-red-...
1,Kanave Kalaiyadhe,(1999),"{'Rating': ' 34% 6.0 ', 'Genres': 'Action & A...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}",{'stream_hd': [{'Amazon Prime Video': 'Subs HD...,{'stream_sd': [{'Amazon Prime Video': 'Subs '}...,"{'stream_free': [], 'rent_free': [], 'buy_free...",{'stream_best': [{'Amazon Prime Video': 'Subs ...,https://www.imdb.com/title/tt9001808/?ref_=ref...,https://www.justwatch.com/us/movie/kanave-kala...
2,Straight Shooter,(1999),"{'Rating': ' 55% 5.0 ', 'Genres': 'Drama , My...",None,None,None,None,None,https://www.imdb.com/title/tt0169272/?ref_=ref...,https://www.justwatch.com/us/movie/straight-sh...
3,Sikandar Sadak Ka,(1999),"{'Rating': ' 24% 4.1 ', 'Genres': 'Action & A...",None,None,None,None,None,https://www.imdb.com/title/tt4995644/?ref_=ref...,https://www.justwatch.com/us/movie/sikandar-sa...
4,Restraining Order,(1999),"{'Rating': ' 46% 4.3 ', 'Genres': 'Action & A...",None,None,None,None,None,https://www.imdb.com/title/tt0169174/?ref_=ref...,https://www.justwatch.com/us/movie/restraining...
...,...,...,...,...,...,...,...,...,...,...
95,El perron de Jalisco,(1999),"{'Rating': ' 29% ', 'Genres': 'Action & Advent...",None,None,None,None,None,None,https://www.justwatch.com/us/movie/el-perron-d...
96,WCW Bash at The Beach 1999,(1999),"{'Rating': ' 80% 5.1 ', 'Genres': 'Action & A...",None,None,None,None,None,https://www.imdb.com/title/tt1224137/?ref_=ref...,https://www.justwatch.com/us/movie/wcw-bash-at...
97,WWE Rebellion 1999,(1999),"{'Rating': ' 64% 6.1 ', 'Genres': 'Drama , Ac...",None,None,None,None,None,https://www.imdb.com/title/tt0403659/?ref_=ref...,https://www.justwatch.com/us/movie/wwe-rebelli...
98,The Silencer,(1999),"{'Rating': ' 54% 5.0 ', 'Genres': 'Action & A...",None,None,None,None,None,https://www.imdb.com/title/tt0262002/?ref_=ref...,https://www.justwatch.com/us/movie/the-silencer


In [ ]:
k_df = df['4k prices'].apply(pd.Series)
h_df = df['hd prices'].apply(pd.Series)
s_df = df['sd prices'].apply(pd.Series)
f_df = df['free'].apply(pd.Series)
b_df = df['best prices'].apply(pd.Series)
d_df = df['detail'].apply(pd.Series)
d_df

,Rating,Genres,Runtime,Age rating,Director
0,88% 8.1 (1m),"Action & Adventure , Fantasy",2h 23min,PG-13,Gore Verbinski


In [ ]:
df = pd.concat([df, k_df.reindex(df.index)], axis=1)
df = pd.concat([df, h_df.reindex(df.index)], axis=1)
df = pd.concat([df, s_df.reindex(df.index)], axis=1)
df = pd.concat([df, f_df.reindex(df.index)], axis=1)
df = pd.concat([df, b_df.reindex(df.index)], axis=1)
df = pd.concat([df, d_df.reindex(df.index)], axis=1)

df_c = df.drop(['4k prices', 'hd prices', 'sd prices', 'free', 'best prices', 'detail'], axis=1)

,title,year,imdb_link,tite url,stream_4k,rent_4k,buy_4k,stream_hd,rent_hd,buy_hd,...,rent_free,buy_free,stream_best,rent_best,buy_best,Rating,Genres,Runtime,Age rating,Director
0,Pirates of the Caribbean: The Curse of the Bl...,(2003),https://www.imdb.com/title/tt0325980/?ref_=ref...,https://www.justwatch.com/us/movie/pirates-of-...,[{'Disney Plus': 'Subs 4K '}],"[{'Apple iTunes': '$3.99 4K '}, {'Amazon Video...","[{'Apple iTunes': '$17.99 4K '}, {'Amazon Vide...",[{'Disney Plus': 'Subs HD '}],"[{'Apple iTunes': '$3.99 HD '}, {'Amazon Video...","[{'Spectrum On Demand': '$3.99 HD '}, {'Apple ...",...,[],[],[{'Disney Plus': 'Subs 4K '}],"[{'Apple iTunes': '$3.99 4K '}, {'Amazon Video...","[{'Spectrum On Demand': '$3.99 HD '}, {'AMC on...",88% 8.1 (1m),"Action & Adventure , Fantasy",2h 23min,PG-13,Gore Verbinski


In [ ]:
df.to_csv("2022_action.csv")